In [1]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

# We store our module in ../src so we need to help python find it
import sys, os
sys.path.insert(1, os.path.join(sys.path[0], '..'))

# Render images directly in the notebook
%matplotlib notebook

# Jupyter gets confused by our module location and won't automatically reload it,
# so we use the autoreload extension to help him out
%load_ext autoreload
%autoreload 2

### Load pneumonia cohort

In [2]:
from src.data import Cohort

root_dir = '/home/goodwintrr/cebmimic/travis/amia_summit_2019/data/'
experiment_dir = root_dir + 'medtagger_filtered/'
cohort = Cohort.from_chronologies(os.path.join(root_dir, experiment_dir, 
                                               'pneumonia.chronology.tsv'),
                                  os.path.join(root_dir, experiment_dir, 
                                               'pneumonia.vocabulary.tsv'))

### Create training, development, and testing splits
We use a 8:1:1 ratio, and split at the *patient*-level, rather than the chronology/admission- level

In [3]:
from sklearn.model_selection import train_test_split

# Sklearn doesn't have a handy way to do a 3-way split, so we do it in two steps:
# First, split to 80% training, 20% testing
train_devel, test = train_test_split(cohort.patients(), test_size=0.1)

# Second, split train into train and devel
train, devel = train_test_split(train_devel, test_size=0.1 / .9)

del train_devel

### Initialize our CANTRIP model

In [4]:
sys.path

['',
 '..',
 '/home/goodwintrr/opt/anaconda3/envs/tensorflow-1.9/lib/python36.zip',
 '/home/goodwintrr/opt/anaconda3/envs/tensorflow-1.9/lib/python3.6',
 '/home/goodwintrr/opt/anaconda3/envs/tensorflow-1.9/lib/python3.6/lib-dynload',
 '/home/goodwintrr/opt/anaconda3/envs/tensorflow-1.9/lib/python3.6/site-packages',
 '/home/goodwintrr/opt/anaconda3/envs/tensorflow-1.9/lib/python3.6/site-packages/IPython/extensions',
 '/home/goodwintrr/.ipython']

In [4]:
import tensorflow as tf

import importlib
import src
importlib.reload(src)

from src.models import CANTRIP
from src.models.doc import rnn_encoder

import shutil
import time



params = {
    'max_seq_len': 7,
    'max_doc_len': 100,
    'vocabulary_size': len(cohort.vocabulary),
    'embedding_size': 200,
    'num_hidden': 100,
    'batch_size': 200,
    'cell_type': 'LSTM',
    'doc_embedding': rnn_encoder(1)
}

tf.reset_default_graph()

model, optimizer, summarizer = CANTRIP(**params)

if 'sess' in globals():
    print('Closing existing session')
    sess.close()

sess = tf.InteractiveSession()

summaries_dir = '../data/working/summaries'
model_summaries_dir = os.path.join(summaries_dir, params['cell_type'], 'RNN-1')



shutil.rmtree(model_summaries_dir, ignore_errors=True)

summary_writer = tf.summary.FileWriter(model_summaries_dir, sess.graph, session=sess)
                               
sess.run(tf.global_variables_initializer())
sess.run(tf.local_variables_initializer())

/home/goodwintrr/opt/anaconda3/envs/tensorflow-1.9/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


RNN Cell: <tensorflow.python.ops.rnn_cell_impl.BasicLSTMCell object at 0x7fb0c3d38a20>
RNN Initial State: LSTMStateTuple(c=<tf.Tensor 'BasicLSTMCellZeroState/zeros:0' shape=(1400, 200) dtype=float32>, h=<tf.Tensor 'BasicLSTMCellZeroState/zeros_1:0' shape=(1400, 200) dtype=float32>)
RNN Final State: LSTMStateTuple(c=<tf.Tensor 'rnn/while/Exit_3:0' shape=(1400, 200) dtype=float32>, h=<tf.Tensor 'rnn/while/Exit_4:0' shape=(1400, 200) dtype=float32>)
RNN Final State Type: <class 'tensorflow.python.ops.rnn_cell_impl.LSTMStateTuple'>
RNN Output: Tensor("rnn/while/Exit_4:0", shape=(1400, 200), dtype=float32)
RNN Output Type: <class 'tensorflow.python.framework.ops.Tensor'>
Doc Embedding: Tensor("Reshape_2:0", shape=(200, 7, 200), dtype=float32)
Deltas:  Tensor("deltas:0", shape=(200, 7), dtype=float32)
RNN Cell: <tensorflow.python.ops.rnn_cell_impl.BasicLSTMCell object at 0x7fb0c1563ba8>
RNN Initial State: LSTMStateTuple(c=<tf.Tensor 'seq_rnn/BasicLSTMCellZeroState/zeros:0' shape=(200, 100) d

In [ ]:
from tqdm import tqdm, tqdm_notebook, tnrange
from tensorflow.python.client import timeline

TRAINING_EPOCHS = 30

for i in tnrange(TRAINING_EPOCHS, desc='Training'):
    sess.run(summarizer.train.reset_op)
    for j, batch in enumerate(tqdm_notebook(cohort[train].make_epoch_batches(**params), desc='Epoch %d' % (i + 1))):
        _, batch_summary, _, global_step = sess.run([optimizer.train_op, summarizer.batch_summary, summarizer.train.metric_ops, optimizer.global_step], 
                                                    batch.feed(model))
        summary_writer.add_summary(batch_summary, global_step)
        summary_writer.add_summary(sess.run(summarizer.train.summary), global_step)
        
        if (j + 1) % 10 == 0:
            sess.run(summarizer.devel.reset_op)
            for batch in cohort[devel].make_epoch_batches(**params):
                sess.run(summarizer.devel.metric_ops, batch.feed(model))
            summary_writer.add_summary(sess.run(summarizer.devel.summary), global_step)
    
    sess.run(summarizer.test.reset_op)
    for batch in cohort[test].make_epoch_batches(**params):
        sess.run(summarizer.test.metric_ops, batch.feed(model))
    summary_writer.add_summary(sess.run(summarizer.test.summary), global_step)

In [ ]:
d = {'a': 1, 'b': 2, 'c': 3}
e = {'d': 4, 'e': 5, 'f': 6}
print(list({**d, **e, 'k':50}))

In [ ]:
len(5)

In [1]:
import tensorflow as tf
with tf.device('/gpu:0'):
    a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
    b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
    c = tf.matmul(a, b)

with tf.Session() as sess:
    print (sess.run(c))

/home/goodwintrr/opt/anaconda3/envs/tensorflow-1.9/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/goodwintrr/opt/anaconda3/envs/tensorflow-1.9/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


InvalidArgumentError: Cannot assign a device for operation 'MatMul': Operation was explicitly assigned to /device:GPU:0 but available devices are [ /job:localhost/replica:0/task:0/device:CPU:0 ]. Make sure the device specification refers to a valid device.
	 [[Node: MatMul = MatMul[T=DT_FLOAT, transpose_a=false, transpose_b=false, _device="/device:GPU:0"](a, b)]]

Caused by op 'MatMul', defined at:
  File "/home/goodwintrr/opt/anaconda3/envs/tensorflow-1.9/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/goodwintrr/opt/anaconda3/envs/tensorflow-1.9/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/goodwintrr/opt/anaconda3/envs/tensorflow-1.9/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/goodwintrr/opt/anaconda3/envs/tensorflow-1.9/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.s